In [1]:
from operator import itemgetter, attrgetter
import json

with open('./one_epoch_models.json') as f:
    models_params_data = json.load(f)

In [2]:
models_params_data[0]

[0.994,
 {'conv_layers': [{'in_channels': 1, 'out_channels': 8, 'kernel_size': 5},
   {'in_channels': 8, 'out_channels': 16, 'kernel_size': 5}],
  'regular_layers': [256, 128, 64, 10],
  'activation_function': 'Swish',
  'optimizer': 'SGD',
  'criterion': 'CrossEntropy',
  'learning_rate': 0.1,
  'momentum': 0.5}]

In [3]:
print(len(models_params_data))

1296


In [4]:
show_top = 5
for i in range(0, show_top):
    print((models_params_data[i][1]))

{'conv_layers': [{'in_channels': 1, 'out_channels': 8, 'kernel_size': 5}, {'in_channels': 8, 'out_channels': 16, 'kernel_size': 5}], 'regular_layers': [256, 128, 64, 10], 'activation_function': 'Swish', 'optimizer': 'SGD', 'criterion': 'CrossEntropy', 'learning_rate': 0.1, 'momentum': 0.5}
{'conv_layers': [{'in_channels': 1, 'out_channels': 8, 'kernel_size': 5}, {'in_channels': 8, 'out_channels': 16, 'kernel_size': 5}], 'regular_layers': [256, 128, 10], 'activation_function': 'ReLU', 'optimizer': 'SGD', 'criterion': 'CrossEntropy', 'learning_rate': 0.1, 'momentum': 0.5}
{'conv_layers': [{'in_channels': 1, 'out_channels': 8, 'kernel_size': 5}, {'in_channels': 8, 'out_channels': 16, 'kernel_size': 5}], 'regular_layers': [256, 128, 32, 10], 'activation_function': 'Swish', 'optimizer': 'Adam', 'criterion': 'CrossEntropy', 'learning_rate': 0.001, 'momentum': 0}
{'conv_layers': [{'in_channels': 1, 'out_channels': 8, 'kernel_size': 5}, {'in_channels': 8, 'out_channels': 16, 'kernel_size': 5}]

In [5]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
import random
import itertools
import time
import heapq
import gc
import json

In [6]:

# How many images to feed through the network each time
batch_size_train = 64
batch_size_test = 1000

# Step size for gradiant decent
learning_rate = 0.01

# Gives the network a chance to jump out of a local minima
momentum = 0.5

# Use a constant seed for randomness so that reruns becomes predictable
random_seed = 1
random.seed(random_seed)
torch.manual_seed(random_seed)

In [7]:
training_set = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                             torchvision.transforms.Normalize((0.5,), (0.5,))])),batch_size=batch_size_train, shuffle=True, num_workers=4, pin_memory=True)

test_set = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.5,), (0.5,))])),batch_size=batch_size_test, shuffle=True, num_workers=4, pin_memory=True)

In [8]:

class Model(nn.Module):
    def __init__(self, hyper_params):
        super(Model, self).__init__()
        self.hyper_params = hyper_params
        
        layers = []
        for n in self.hyper_params['conv_layers']:
            conv_layer = nn.Conv2d(
                n['in_channels'], 
                n['out_channels'], 
                n['kernel_size']
            )
            layers.append(conv_layer)
            layers.append(self.get_activation_function())
            layers.append(nn.MaxPool2d(2))
        self.conv = nn.Sequential(*layers)
        

        layers = []
        for i in range(len(self.hyper_params['regular_layers']) - 1):
            nr_in = self.hyper_params['regular_layers'][i];
            nr_out = self.hyper_params['regular_layers'][i + 1];
            layers.append(nn.Linear(nr_in, nr_out))
            layers.append(self.get_activation_function())
        
        # The output layer should not have an activation function
        layers.pop()
        
        self.net = nn.Sequential(*layers)
        
        
        learning_rate = self.hyper_params['learning_rate']
        momentum = self.hyper_params['momentum']
        optimizer = self.hyper_params['optimizer']
        if optimizer == 'SGD':
            self.optimizer = optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)
        elif optimizer == 'Adam':
            self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        else:
            raise Exception('Invalid optimizer')
            
        criterion = self.hyper_params['criterion']
        if criterion == 'CrossEntropy':
            self.criterion = nn.CrossEntropyLoss()
        else:
            raise Exception('Invalid criterion')
        
        
    def get_activation_function(self):
        fun = self.hyper_params['activation_function']
        if fun == 'ReLU':
            return nn.ReLU()
        if fun == 'Sigmoid':
            return torch.nn.Sigmoid()
        if fun == 'Swish':
            return Swish()
        
        
    def forward(self, tensor):
        tensor = self.conv(tensor)
        tensor = tensor.reshape(tensor.size(0), -1)
        return self.net(tensor)
    
    
    def fit(self, tensor, labels):
       
        result = self(tensor)
        
        self.optimizer.zero_grad()
        loss = self.criterion(result, labels)
        loss.backward()
        self.optimizer.step()
        
        _, predictions = torch.max(result, 1)
        total = tensor.size(0)
        nr_correct = torch.sum(predictions == labels).item()
        
        return nr_correct / total
    
    
    #Needed to not compare models stored in a tuple
    def __lt__(self, m):
        return False
    
    
class Swish(nn.Module):
    def forward(self, input_tensor):
        return input_tensor * torch.sigmoid(input_tensor)
        

In [9]:
def train_model(model):
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    total = 0
    nr_correct = 0

    accuracies = []
    for (batch_index, (images, labels)) in enumerate(training_set):
        accuracy = model.fit(images, labels)
        accuracies.append(accuracy)


    return accuracies

In [10]:
def test_model(model, test_set):
    total = 0
    nr_correct = 0
    
    for images, labels in test_set:
        
        result = model(images)
        
        _, predictions = torch.max(result, 1)
        total = images.size(0)
        nr_correct = torch.sum(predictions == labels).item()
                
    return nr_correct / total

In [ ]:
top_models_num = 10
epochs = 20
models_accuracies = {}

time_start = time.time()
model_id = 0

for model_data in models_params_data[:top_models_num]:
    
    model_hyper_params = model_data[1] # select the dictionary with the yper params for each model
    models_accuracies[model_id] = []
    
    model = Model(model_hyper_params)
    print("Training and Testing for model with id: ", model_id)

    for epoch in range(1, epochs+1):
        
        train_model(model)
        
        accuracy = test_model(model, test_set)
        
        models_accuracies[model_id].append(accuracy)
        print(accuracy)
        
        
    model_id+=1

#     print(models_accuracies)
#     print("Training and Testing completed for model with id: ", model_id)

    

Training and Testing for model with id:  0
0.981
0.981
0.979
0.987
0.985
0.991
0.99
0.986
0.984
0.991
0.993
0.993
0.99
0.996
0.995
0.989
0.997
0.99
0.992
0.991
Training and Testing for model with id:  1
0.973
0.986
0.981
0.986
0.993
0.986
0.983
0.987
0.987
0.992
0.997
0.988
0.986
0.989
0.987
0.982
0.993
0.994
0.991
0.992
Training and Testing for model with id:  2
0.965
0.986
0.986
0.987
0.995
0.994


In [ ]:
print(models_accuracies)

In [ ]:
# {0: [0.981, 0.984, 0.986, 0.985, 0.987, 0.99, 0.988, 0.985, 0.986, 0.988, 0.99, 0.988, 0.996]}


In [ ]:
params_json = json.dumps(models_accuracies)
f = open("top_models_accuracies", "w")
f.write(params_json)
f.close()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epoch_range = list(range(1, epochs_+1, 1))

colors = ['#04f108', '#f11f04', '#f15704', '#e5f104', '#7af104', '#04f186', '#b2f104', '#043bf1', '#c204f1', 
          '#f104be', '#f10404']


fig, ax = plt.subplots()
plt.title("Top 10 Models Accuracy Scores")

plt.fig(figsize=(5,6))

for i in range(epochs):
    ax.plot(epoch_range, models_accuracies[i], colors[i], linewidth=1, label="Model "+str(i+1))
#     ax.plot(t,models_accuracies[1], "b-", linewidth=1, label="Model 2")
    
ax.legend()
ax.set_ylabel("Accuracy Score")
ax.set_xlabel("Number of Epochs")
plt.show()

 